In [17]:
import pandas as pd
import torch
import sys
import numpy as np
from numpy.linalg import eigvals, matrix_rank
sys.path.append("../utility")
from network import KoopmanNet

project_name = "May_3_residual"

In [24]:
log = pd.read_csv(f"../log/{project_name}/koopman_results_log.csv")
log

,env_name,encode_dim,hidden_layers,hidden_dim,seed,normalize,best_val_Kloss,convergence_val_Kloss,test_Kloss,test_CovLoss,test_ControlLoss,model_path,num_params,encoder_num_params
0,Polynomial,64,3,256,1,True,0.000009,0.000010,0.000009,1657.320801,NaN,../log/May_3_residual/best_models/20250502_193...,171017,166528
1,Polynomial,64,3,256,1,True,0.000011,0.000012,0.000011,140.440903,NaN,../log/May_3_residual/best_models/20250502_193...,153545,149056
2,LogisticMap,64,3,256,1,True,0.375360,0.375578,0.375786,1666.445801,NaN,../log/May_3_residual/best_models/20250502_193...,169729,165504
3,LogisticMap,64,3,256,1,True,0.330051,0.330116,0.330845,16.867357,NaN,../log/May_3_residual/best_models/20250502_194...,152769,148544
4,DampingPendulum,64,3,256,1,True,0.005690,0.005698,0.005739,3101.483398,0.995975,../log/May_3_residual/best_models/20250502_194...,170438,166016
5,DampingPendulum,64,3,256,1,True,0.005717,0.005721,0.005739,36.271774,0.995364,../log/May_3_residual/best_models/20250502_194...,153222,148800
6,DoublePendulum,64,3,256,1,True,0.145875,0.145906,0.146991,142.428772,0.881567,../log/May_3_residual/best_models/20250502_194...,171800,167040
7,DoublePendulum,64,3,256,1,True,0.146026,0.146055,0.147176,28.694832,0.881287,../log/May_3_residual/best_models/20250502_195...,154072,149312
8,Franka,64,3,256,1,True,0.000019,0.000019,0.000018,75.687576,0.000001,../log/May_3_residual/best_models/20250502_195...,180824,173696
9,Franka,64,3,256,1,True,0.000021,0.000021,0.000020,15.093703,0.000003,../log/May_3_residual/best_models/20250502_195...,159768,152640


In [26]:
ckpt_path = log.iloc[5].model_path
ckpt = torch.load(ckpt_path)
layers = ckpt.get("encode_layers", ckpt["layer"])
feature_dim = layers[-1]
Nkoop = 2 + 64
control_dim = 1
net = KoopmanNet(layers, Nkoop, control_dim, use_residual=False)
net.load_state_dict(ckpt.get("state_dict", ckpt["model"]))
net.eval()
A = net.lA.weight.detach().numpy()
B = net.lB.weight.detach().numpy()

In [27]:
#  Generate controllability matrix C_n = [B, AB, A^2B, ..., A^{n-1}B]
n = A.shape[0]
C_n = B
for i in range(1, n):
    C_n = np.hstack((C_n, np.linalg.matrix_power(A, i) @ B))

print("Controllability matrix C_n:\n", C_n)

# Compute eigenvalues of controllability matrix (C_n @ C_n^T for square matrix)
C_square = C_n @ C_n.T
eig_C = eigvals(C_square)

#print("Eigenvalues of C_n @ C_n.T:", eig_C)
print("Max eigenvalue of C_n:", np.max(np.abs(eig_C)))
print("Min eigenvalue of C_n:", np.min(np.abs(eig_C)))

Controllability matrix C_n:
 [[ 5.2658527e-04  7.9209078e-04  8.0390740e-04 ...  4.5830682e-03
   4.7423299e-03  4.2951470e-03]
 [ 7.4306196e-03  5.6679174e-03  2.7161445e-03 ...  9.8989327e-03
  -3.8184924e-03 -1.7456714e-02]
 [ 8.7602717e-01 -2.6012892e-01 -2.0136182e-01 ...  3.7360881e-03
   6.0563050e-03  8.1825498e-03]
 ...
 [ 3.8529211e-01  2.3739581e-01 -4.0555817e-01 ...  3.0254626e-03
   3.7166707e-03  4.3428764e-03]
 [ 5.5308435e-02 -2.1487078e-02 -3.6993034e-02 ... -1.2060626e-03
  -3.3542996e-03 -5.0488785e-03]
 [-9.2397429e-02  3.2156190e-01 -4.2030837e-02 ... -5.1623560e-02
  -4.2162865e-02 -2.9358137e-02]]
Max eigenvalue of C_n: 26.14686
Min eigenvalue of C_n: 1.6889183e-08
